In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd
import numpy as np

import re
import codecs
# You need to download the nltk data by calling the following script
# import nltk
# nltk.download('stopwords')
# and then selecting where to install the data (usually 'C:\nltk:data')

def text_cleaner(text):
    rules = [
        {r'>\s+': u'>'},  # remove spaces after a tag opens or closes
        {r'\s+': u' '},  # replace consecutive spaces
        {r'\s*<br\s*/?>\s*': u'\n'},  # newline after a <br>
        {r'</(div)\s*>\s*': u'\n'},  # newline after </p> and </div> and <h1/>...
        {r'</(p|h\d)\s*>\s*': u'\n\n'},  # newline after </p> and </div> and <h1/>...
        {r'<head>.*<\s*(/head|body)[^>]*>': u''},  # remove <head> to </head>
        {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},  # show links instead of texts
        {r'[ \t]*<[^<]*?/?>': u''},  # remove remaining tags
        {r'^\s+': u''}  # remove spaces at the beginning
    ]
    for rule in rules:
        for (k, v) in rule.items():
            regex = re.compile(k)
            text = regex.sub(v, text)
    text = text.strip()
    return text.lower()

In [2]:
#adds path to dataframe

#change path according to your system

#path to master996.csv
csv = pd.read_csv("D:/Uni/Master 1/Advanced Topics in ML/Programming Assignment/Gutenberg_English_Fiction_1k/master996.csv",encoding='windows-1252',sep=';')
#path to html (text) directory
content_dir = "D:/Uni/Master 1/Advanced Topics in ML/Programming Assignment/Gutenberg_English_Fiction_1k/Gutenberg_19th_century_English_Fiction/"
                                      
book_ids = csv.book_id

def id_to_path(id):
    return id[:-5] + "-content.html"

path_list=[]
for id in book_ids:
    path_list.append(content_dir + id_to_path(id))
    
csv['path'] = path_list

csv

,Book_Name,book_id,guten_genre,Author_Name,path
0,The Mystery of the Boule Cabinet: A Detective ...,pg10067.epub,Detective and Mystery,Stevenson| Burton Egbert,D:/Uni/Master 1/Advanced Topics in ML/Programm...
1,The Pupil,pg1032.epub,Literary,James| Henry,D:/Uni/Master 1/Advanced Topics in ML/Programm...
2,At Love's Cost,pg10379.epub,Literary,Garvice| Charles,D:/Uni/Master 1/Advanced Topics in ML/Programm...
3,The Heart of the Range,pg10473.epub,Western Stories,White| William Patterson,D:/Uni/Master 1/Advanced Topics in ML/Programm...
4,The Worshipper of the Image,pg10812.epub,Literary,Gallienne| Richard Le,D:/Uni/Master 1/Advanced Topics in ML/Programm...
...,...,...,...,...,...
991,David Copperfield,pg766DickensDavidCopfld.epub,Literary,Dickens| Charles,D:/Uni/Master 1/Advanced Topics in ML/Programm...
992,Hard Times,pg786DickensHardTimes.epub,Literary,Dickens| Charles,D:/Uni/Master 1/Advanced Topics in ML/Programm...
993,Memoirs of Shelock Holmes,pg834DoyleMemoirsSherlk.epub,Detective and Mystery,Connan| Doyle,D:/Uni/Master 1/Advanced Topics in ML/Programm...
994,The Mysterious Affair at Styles,pg863Agatha1.epub,Detective and Mystery,Christie| Agatha,D:/Uni/Master 1/Advanced Topics in ML/Programm...


In [3]:
import string
from nltk.corpus import stopwords
import os
import glob


# remove could and would
stop_words = set(stopwords.words('english'))
stop_words.add('could') 
stop_words.add('would')
stop_words.add('p')

#Tokenization
from nltk.tokenize import word_tokenize

from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()
#remove html tags

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

for index, row in csv.iterrows():
    #remove punctuation
    content = open(csv.path[index],encoding="utf8").read()
    content = content.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    content = text_cleaner(content)
    tokens = word_tokenize(content)
    #stopword removal
    filtered_text = [w for w in tokens if not w in stop_words]
    #stemming
    for w in filtered_text:
    #        w = ps.stem(w)
    #lemmatization
            w = lemmatizer.lemmatize(w)            
    outfile = open('D:/Uni/Master 1/Advanced Topics in ML/Programming Assignment/Gutenberg_English_Fiction_1k/1 tokenized books/'+ csv.book_id[index] + '_tokenized.html', "a", encoding='utf-8')
    for w in filtered_text:
        outfile.write(w+",")

In [ ]:
#list all characters

import spacy

sp = spacy.load('en_core_web_sm')

for index, row in csv.iterrows():
    tupleList = []
    entity_set = set()
    text = open(csv.path[index],"r", encoding='utf-8').read()
    text = text_cleaner(text)
    text = text.replace('"','')
    text = text.replace('?','? ')
    text = text.replace('.','. ')
    text = text.replace(',',', ')
    text = text.replace('!','! ')
    sentences = sp(text)
    for sentence in sentences.sents:
        for entity in sentence.ents:
            if entity.label_ in ["PERSON"]:
                entity_set.add(entity.text)
                
    for name in entity_set:
        namecount = 0
        namecount = text.count(name)
        if(namecount > 10):
            tupleList.append((name,namecount))
        
    outfile = open('D:/Uni/Master 1/Advanced Topics in ML/Programming Assignment/Gutenberg_English_Fiction_1k/entity_lists/'+ csv.book_id[index] + '_entity_list.html', "a", encoding='utf-8')
    for w in tupleList:
        outfile.write(str(w))
        outfile.write(",\n")
        